In [1]:
import os

# from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatVertexAI
from langchain.memory import ConversationBufferMemory

from chatrag.retriever import create_retrieval_chain, create_retriever_from_csv

In [2]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "chatrag"
os.environ["WANDB_NOTEBOOK_NAME"] = "/Users/itortouch/GiTor/chat_rag/notebooks/retriever_agent_react_poc_gcp.ipynb"

# GOOGLE AUTH FLOW
Elige uno

## Opcion 1. OAuth

### sub option 1.1 From file

In [ ]:
# import os.path

# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow

# SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

# def load_creds():
#     """Converts `credentials.json` to a credential object.

#     This function caches the generated tokens to minimize the use of the
#     consent screen.
#     """
#     creds = None
#     # The file token.json stores the user's access and refresh tokens, and is
#     # created automatically when the authorization flow completes for the first
#     # time.
#     if os.path.exists('token.json'):
#         creds = Credentials.from_authorized_user_file('token.json', SCOPES)
#     # If there are no (valid) credentials available, let the user log in.
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file(
#                 'credentials.json', SCOPES)
#             creds = flow.run_local_server(port=0)
#         # Save the credentials for the next run
#         with open('token.json', 'w') as token:
#             token.write(creds.to_json())
#     return creds

In [ ]:
# creds = load_creds()

In [ ]:
# from langchain.utilities.vertexai import init_vertexai
# init_vertexai(project="chatrag", location="europe-west9", credentials=creds)

In [ ]:
# from vertexai.language_models import ChatModel, InputOutputTextPair
# chat_model = ChatModel.from_pretrained("chat-bison")
# parameters = {
#     "candidate_count": 1,
#     "max_output_tokens": 1024,
#     "temperature": 0.2,
#     "top_p": 0.8,
#     "top_k": 40
# }
# chat = chat_model.start_chat()
# response = chat.send_message("""hola""", **parameters)
# print(f"Response from Model: {response.text}")

### sub option 1.2 From env var

In [ ]:
# import os.path

# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow

# SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

# def load_creds():
#     """Converts `gcp_chatrag_client_config env var` to a credential object."""
#     creds = None
#     # If there are no (valid) credentials available, let the user log in.
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_config(eval(st.secrets["gcp_chatrag_client_config"]), SCOPES)
#             creds = flow.run_local_server(port=0)



In [ ]:
# from langchain.utilities.vertexai import init_vertexai
# init_vertexai(project="chatrag", location="europe-west9", credentials=creds)

In [ ]:
# retrieval_chain_llm = ChatVertexAI(project="chatrag", location="europe-west9", model_name="chat-bison", temperature=0.7, max_output_tokens=2000)
# retrieval_chain_llm.predict("Hola")

### sub option 1.3 From Flow

In [3]:
import webbrowser

In [4]:
from google_auth_oauthlib.flow import Flow

SCOPES = ['https://www.googleapis.com/auth/cloud-platform']
# Create the flow using the client secrets file from the Google API
# Console.
flow = Flow.from_client_secrets_file(
    # '../data/gcp-service-account-key.json',
    '../data/gcp-oauth-keys.json',
    scopes=SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob') #?

# Tell the user to go to the authorization URL.
auth_url, _ = flow.authorization_url(prompt='consent') #?
webbrowser.open_new_tab(auth_url)

# print('Please go to this URL: {}'.format(auth_url))

# The user will get an authorization code. This code is used to get the
# access token.
code = input('Enter the authorization code: ')
flow.fetch_token(code=code)

# You can use flow.credentials, or you can just get a requests session
# using flow.authorized_session.
# session = flow.authorized_session()
# print(session.get('https://www.googleapis.com/userinfo/v2/me').json())

Please go to this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=36287955814-5d96jpso68akjrhv704624a4prqubri8.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=OG4H9bCbuYkDPLY1bip952L65W2C1l&prompt=consent&access_type=offline
here


In [5]:
from langchain.utilities.vertexai import init_vertexai
init_vertexai(project="chatrag", location="europe-west9", credentials=flow.credentials)

In [ ]:
retrieval_chain_llm = ChatVertexAI(project="chatrag", location="europe-west9", model_name="chat-bison", temperature=0.7, max_output_tokens=2000)

In [ ]:
retrieval_chain_llm.predict("Hola")

## Opcion 2. Env Var

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../data/gcp-secrets.json"

## Data preparation

In [ ]:
retriever_llm = ChatVertexAI(project="chatrag", location="europe-west9", model_name="chat-bison", temperature=0.7, max_output_tokens=2000)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"vote_average": "float"},
    llm=retriever_llm
)

In [ ]:
# retriever.get_relevant_documents("I want to watch a movie about space battles.")

# Vectorstore DB

In [ ]:
retrieval_chain_llm = ChatVertexAI(project="chatrag", location="europe-west9", model_name="chat-bison", temperature=0.7, max_output_tokens=2000)
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

In [ ]:
# print(g_media_sq_retrieval_chain.run("I want to watch a movie about space battles."))

# Tooling

In [ ]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence."""

In [ ]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    )
]

# Prompt templates

In [ ]:
# flake8: noqa
PREFIX = """You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.

Take a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS."""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_kwargs = {
        "system_message": PREFIX,
    }

# Chains

In [ ]:
agent_llm_model = ChatVertexAI(project="chatrag", location="europe-west9", model_name="chat-bison", temperature=0.7, max_output_tokens=2000)

In [ ]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True
)

# TESTS

### Run

In [ ]:
from langchain.globals import set_debug
set_debug(True)

In [ ]:
response = general_chat_agent.run(input="Hi, my name is Aitor")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="""I would like to watch a movie""")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="""I want to watch a movie about outer space exploration.""")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="I've changed my mind. Would you suggest me a good genre for just relaxing?.")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="By the way, what's your name?.")
print("\nA partir de aquí la respuesta:\n")
print(response)